In [4]:
import os
import sys
import pandas as pd
module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

from my_utils.util_minio import MinioHandler  # MinioHandler is a class in util_minio module
import config


In [11]:

class DataTransformer:
    def __init__(self, bucket_name, file_path):
        self.bucket_name = bucket_name
        self.file_path = file_path
        self.dict_storage_options = config.MINIO_CONFIG
    
    def download_to_dataframe(self):
        minio_handle = MinioHandler()  # Assuming MinioHandler is properly implemented
        return minio_handle.download_to_dataframe(self.bucket_name, self.file_path)

    def save_to_parquet(self, df, partition_cols, cols_dim):
        df_dim_product = df[cols_dim]
        df_dim_product.to_parquet('s3://tiki/curated/dim_product', 
                                  storage_options=self.dict_storage_options, 
                                  partition_cols=partition_cols,
                                  ) #TODO: add append to false

# Define input parameters
bucket_name = config.BUCKET_CONFIG
file_path = 'raw/raw_116532_1711993659.csv'

partition_cols = ['ingestion_date']
cols_dim = ["tiki_pid","name","brand_name","origin",'ingestion_date','ingestion_dt_unix']  # Define the columns here

# Create instance of DataHandler
data_handler = DataTransformer(bucket_name, file_path)

# Download data to DataFrame
df = data_handler.download_to_dataframe()

# Save DataFrame to Parquet
data_handler.save_to_parquet(df, partition_cols=partition_cols, cols_dim=cols_dim)

,tiki_pid,sku,name,url_key,url_path,price,discount,discount_rate,rating_average,review_count,...,seller_type,is_imported,video_url,layout_type,fastest_delivery_time,earliest_delivery_estimate,order_route,imported,ingestion_dt_unix,ingestion_date
0,252608882,4825599512084,Tã/bỉm quần HUGGIES SKINCARE MEGA JUMBO size X...,thung-ta-bim-quan-huggies-skincare-mega-jumbo-...,thung-ta-bim-quan-huggies-skincare-mega-jumbo-...,365000,280750,43,4.8,331,...,OFFICIAL_STORE,False,NaN,NaN,NaN,NaN,NaN,NaN,1712423984,2024-04-06
1,252608891,1033225203769,Tã/bỉm quần HUGGIES SKINCARE gói SUPER JUMBO s...,ta-bim-quan-huggies-skincare-goi-super-jumbo-s...,ta-bim-quan-huggies-skincare-goi-super-jumbo-s...,289000,184130,39,5.0,171,...,OFFICIAL_STORE,False,NaN,NaN,NaN,NaN,NaN,NaN,1712423984,2024-04-06
